In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('jay_1999_game_info.csv')
df.head()

,game_id,game_order,event,site,date_played,round,white,black,result,white_elo,...,winner_loser_elo_diff,eco,termination,time_control,utc_date,utc_time,variant,ply_count,date_created,file_name
0,b1de229e-95a6-45f6-a26c-7fc2d1ac1506,1,Rated Blitz game,https://lichess.org/dnSCO0Cr,2023.02.07,?,Andrea_Tutone,Jay_1999,1-0,1987,...,-56,B01,Normal,180+2,2023.02.07,19:39:01,Standard,NaN,2023-02-10T20:50:53+0000,lichess_Jay_1999_2023-02-10.pgn
1,4b846301-9dc7-4a7a-9816-8afab9145665,2,Rated Blitz game,https://lichess.org/t2z2WkNp,2023.02.07,?,Jay_1999,PotatoMatey,0-1,2050,...,-59,A01,Normal,180+2,2023.02.07,04:54:30,Standard,NaN,2023-02-10T20:50:53+0000,lichess_Jay_1999_2023-02-10.pgn
2,3d1f3dfc-0bfb-43b9-afc1-02269d3753d9,3,Rated Blitz game,https://lichess.org/vxqs7HvO,2023.02.06,?,Jay_1999,Alrrash,0-1,2055,...,71,A01,Normal,180+2,2023.02.06,04:30:26,Standard,NaN,2023-02-10T20:50:53+0000,lichess_Jay_1999_2023-02-10.pgn
3,29fd5ad8-47a7-4b12-a9d3-5548539c0d88,4,Rated Blitz game,https://lichess.org/MlIGVPO8,2023.02.05,?,Crocodingo,Jay_1999,1-0,2115,...,55,E90,Normal,180+2,2023.02.05,23:38:06,Standard,NaN,2023-02-10T20:50:53+0000,lichess_Jay_1999_2023-02-10.pgn
4,1f9ecf78-8bee-44b6-8e02-ebd1e5d349c9,5,Rated Blitz game,https://lichess.org/Vot0ijSw,2023.02.05,?,pijuan,Jay_1999,0-1,2054,...,0,B01,Normal,180+2,2023.02.05,22:09:54,Standard,NaN,2023-02-10T20:50:53+0000,lichess_Jay_1999_2023-02-10.pgn


In [3]:
df.columns

Index(['game_id', 'game_order', 'event', 'site', 'date_played', 'round',
       'white', 'black', 'result', 'white_elo', 'white_rating_diff',
       'black_elo', 'black_rating_diff', 'white_title', 'black_title',
       'winner', 'winner_elo', 'loser', 'loser_elo', 'winner_loser_elo_diff',
       'eco', 'termination', 'time_control', 'utc_date', 'utc_time', 'variant',
       'ply_count', 'date_created', 'file_name'],
      dtype='object')

In [4]:
df = df.drop(columns=['game_id', 'game_order', 'site', 'round', 'white_title', 'black_title', 'ply_count', 'date_created', 'file_name'] )

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2911 entries, 0 to 2910
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   event                  2911 non-null   object 
 1   date_played            2911 non-null   object 
 2   white                  2911 non-null   object 
 3   black                  2911 non-null   object 
 4   result                 2911 non-null   object 
 5   white_elo              2911 non-null   object 
 6   white_rating_diff      2658 non-null   float64
 7   black_elo              2911 non-null   object 
 8   black_rating_diff      2658 non-null   float64
 9   winner                 2911 non-null   object 
 10  winner_elo             2795 non-null   object 
 11  loser                  2911 non-null   object 
 12  loser_elo              2795 non-null   object 
 13  winner_loser_elo_diff  2911 non-null   int64  
 14  eco                    2911 non-null   object 
 15  term

In [6]:
df['white_rating_diff'].fillna('0', inplace=True)
df['black_rating_diff'].fillna('0', inplace=True)
df['winner_elo'].fillna('0', inplace=True)
df['loser_elo'].fillna('0', inplace=True)

In [7]:
def classify_time_control(tc):
    tc_minutes = int(tc.split("+")[0])
    if tc_minutes < 180:
        return "Bullet"
    elif tc_minutes >= 180 and tc_minutes < 310:
        return "Blitz"
    elif tc_minutes >= 310 and tc_minutes <= 900:
        return "Rapid"
    else:
        return "Classical"

df['time_control'] = df['time_control'].apply(classify_time_control)

In [8]:
df['time_control'].value_counts()

blitz        2239
rapid         394
bullet        246
classical      32
Name: time_control, dtype: int64

In [9]:
df['result'] = np.where((df['white'] == 'Jay_1999') & (df['result'] == '1-0'), 'Won', 
                np.where((df['black'] == 'Jay_1999') & (df['result'] == '0-1'), 'Won', 
                np.where(df['result'] == '1/2-1/2', 'Draw', 'Lost')))

In [10]:
df['result'].value_counts()

Won     1596
Lost    1199
Draw     116
Name: result, dtype: int64

In [11]:
def opening_name(eco_code):
    if eco_code in ['A01', 'A06']:
        return 'Nimzovich-Larsen Attack'
    elif eco_code in ['A00']:
        return 'Benko Opening'
    elif eco_code in ['A02', 'A03']:
        return "Bird's Opening"
    elif eco_code in ['A04', 'A05', 'A07', 'A08', 'A09']:
        return 'Reti Opening'
    elif eco_code in ['A10', 'A15', 'A17', 'A13', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28']:
        return 'English Opening'
    elif eco_code in ['A40', 'A41', 'A45', 'A46', 'A50', 'D00', 'D04', 'D05', 'E00', 'E10']:
        return "Queen's Pawn' Opening"
    elif eco_code in ['A43', 'A44']:
        return 'Old Benoni Defense'
    elif eco_code in ['A47']:
        return "Queen's Indian Defense"
    
    elif eco_code in ['A48', 'A49']:
        return "King's Indian, East Indian Defense" 
    elif eco_code in ['A51', 'A52']:
        return "Budapest Defense"
    elif eco_code in ['A53', 'A54', 'A55']:
        return "Old Indian Defense"  
    elif eco_code in ['A57', 'A58', 'A59']:
        return "Benko Gambit" 
    elif eco_code in ['A56', 'A57','A58','A59','A60',
                        'A61','A62','A63','A64','A65','A66','A68','A67','A69','A70',
                        'A71','A72','A73','A74','A75','A77','A76','A78','A79']:
        return "Benoni Defense"  
    elif eco_code in ['A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A90', 
                        'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97', 'A98', 'A99']:
        return "Benko Gambit" 
    elif eco_code in ['B01', 'B02']:
        return 'Scandinavian Defense'
    elif eco_code in ['B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19']:
        return 'Caro-Kann Defense'    
    elif eco_code in ['B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 
                    'B30', 'B31', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39', 
                    'B40', 'B41', 'B42', 'B43', 'B44', 'B45', 'B46', 'B47', 'B48', 'B49', 
                    'B50', 'B51', 'B52', 'B53', 'B54', 'B55', 'B56', 'B57', 'B58', 'B59', 
                    'B60', 'B61', 'B62', 'B63', 'B64', 'B65', 'B66', 'B67', 'B68', 'B69', 
                    'B70', 'B71', 'B72', 'B73', 'B74', 'B75', 'B76', 'B77', 'B78', 'B79', 
                    'B80', 'B81', 'B82', 'B83', 'B84', 'B85', 'B86', 'B87', 'B88', 'B89', 
                    'B90', 'B91', 'B92', 'B93', 'B94', 'B95', 'B96', 'B97', 'B98', 'B99'
                    ]:
        return 'Sicillian Defense'
  
    elif eco_code in ['C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 
                        'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19']:
        return 'French Defense'
        
    elif eco_code in ["C20", "C21", "C22", "C23", "C24", "C25", "C26", "C27", "C28", "C29"]:
        return "King's Pawn Opening" 
    elif eco_code in ["C30", "C31", "C32", "C33", "C34", "C35", "C36", "C37", "C38", "C39"]:
        return "King's Gambit"
    elif eco_code in ['C40']:
        return "King's Knight Opening"
    elif eco_code in ['C41']:
        return "Philidor's Defense"
    elif eco_code in ['C42', 'C43']:
        return "Petrov's Defense"
    elif eco_code in ['C44']:
        return "King's Pawn Opening"
    elif eco_code in ['C45']:
        return "Scotch Opening"
    elif eco_code in ['C46']:
        return "Three Knights Game"
    elif eco_code in ['C47', 'C48', 'C49']:
        return "Four Knights, Scotch Variation"
    elif eco_code in ['C50']:
        return "King's Pawn Game"
    elif eco_code in ['C51', 'C52']:
        return "Evans Gambit"
    elif eco_code in ['C53', 'C54']:
        return "Giuoco Piano"
    elif eco_code in ['C55', 'C56', 'C57', 'C58', 'C59']:
        return "Two Knights Defense"
    elif eco_code in ['C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 
                    'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 
                    'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 
                    'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99']:
        return "Ruy Lopez (Spanish Opening)"
    elif eco_code in ['D02']:
        return 'London System'
    elif eco_code in ['D16', 'D17', 'D18', 'D19', 'D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42']:
        return "Queen's Gambit Declined"
    elif eco_code in ['D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29']:
        return "Queen's Gambit Accepted"
    elif eco_code in ["D80", "D81", "D82", "D83", "D84", "D85", "D86", "D87", "D88", "D89",
                    "D90", "D91", "D92", "D93", "D94", "D95", "D96", "D97", "D98", "D99"]:
        return "Gruenfeld Defense"
    elif eco_code in  ['E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29',     
                    'E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39',     
                    'E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49',
                    'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59']:
        return "Nimzo-Indian Defense"
    elif eco_code in ['E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69',  
                    'E70', 'E71', 'E72', 'E73', 'E74', 'E75', 'E76', 'E77', 'E78', 'E79',  
                    'E80', 'E81', 'E82', 'E83', 'E84', 'E85', 'E86', 'E87', 'E88', 'E89',  
                    'E90', 'E91', 'E92', 'E93', 'E94', 'E95', 'E96', 'E97', 'E98', 'E99']:
        return "King's Indian Defense"
    elif eco_code in ['?']:
        return 'Unknown'
    else:
        return 'Mixed of opening I usually not played'
        


df['Opening'] = df['eco'].apply(opening_name)

In [12]:
df['Opening'].value_counts()

Caro-Kann Defense                        465
Sicillian Defense                        426
Queen's Pawn' Opening                    297
Mixed of opening I usually not played    241
Nimzovich-Larsen Attack                  226
Scandinavian Defense                     186
French Defense                           159
London System                            139
Ruy Lopez (Spanish Opening)              114
Queen's Gambit Declined                   67
Reti Opening                              67
Bird's Opening                            58
English Opening                           57
King's Indian Defense                     52
Benko Opening                             49
Unknown                                   39
King's Pawn Opening                       37
King's Indian, East Indian Defense        33
Philidor's Defense                        25
Benko Gambit                              21
Petrov's Defense                          20
King's Gambit                             17
Benoni Def

In [13]:
df['myrating'] = None

In [14]:
df['white_elo'].replace('?', np.nan, inplace=True)
df['black_elo'].replace('?', np.nan, inplace=True)

df['white_elo'] = pd.to_numeric(df['white_elo'], errors='coerce')
df['black_elo'] = pd.to_numeric(df['black_elo'], errors='coerce')

df['white_rating_diff'] = pd.to_numeric(df['white_rating_diff'], errors='coerce')
df['black_rating_diff'] = pd.to_numeric(df['black_rating_diff'], errors='coerce')



In [15]:
for i, row in df.iterrows():
    if row['event'] not in ['Casual Blitz game', 'Casual Rapid game', 'Casual Classical game', 'Casual Bullet game']:
        if row['white'] == 'Jay_1999' and row['result'] == 'Won':
            df.at[i, 'myrating'] = row['white_elo'] + row['white_rating_diff']
        elif row['black'] == 'Jay_1999' and row['result'] == 'Won':
            df.at[i, 'myrating'] = row['black_elo'] + row['black_rating_diff']
        elif row['white'] == 'Jay_1999' and row['result'] == 'Lost':
            df.at[i, 'myrating'] = row['white_elo']
        elif row['black'] == 'Jay_1999' and row['result'] == 'Lost':
            df.at[i, 'myrating'] = row['black_elo']

In [16]:
df['utc_time'] = pd.to_datetime(df['utc_time'])

In [17]:
df['hour'] = df['utc_time'].dt.hour
df['time_of_day'] = np.where((df['hour'] >= 6) & (df['hour'] < 12), 'Morning',
                   np.where((df['hour'] >= 12) & (df['hour'] < 18), 'Afternoon',
                            np.where((df['hour'] >= 18) & (df['hour'] < 22), 'Evening', 'Night')))


In [18]:
df['termination'].replace("Normal", "Resignation/Checkmate", inplace=True)

In [19]:
df['termination'].value_counts()

Resignation/Checkmate    2326
Time forfeit              571
Abandoned                  14
Name: termination, dtype: int64

In [20]:
df['time_of_day'].value_counts()

morning      1257
afternoon    1245
evening       225
night         184
Name: time_of_day, dtype: int64

In [21]:
print(max(df['winner_loser_elo_diff']) - min(df['winner_loser_elo_diff']))

1794


In [22]:
max(df['winner_loser_elo_diff'])

1123

In [23]:
min(df['winner_loser_elo_diff'])

-671

In [24]:
df['type of game'] = np.where((df['hour'] >= 6) & (df['hour'] < 12), 'morning',
                   np.where((df['hour'] >= 12) & (df['hour'] < 18), 'afternoon',
                            np.where((df['hour'] >= 18) & (df['hour'] < 22), 'evening', 'night')))

In [25]:
for i, row in df.iterrows():
    if row['white'] == 'Jay_1999' and row['result'] == 'Won':
        if row['winner_loser_elo_diff'] >= 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against very high rated player'
        elif row['winner_loser_elo_diff'] >= 50 and row['winner_loser_elo_diff'] < 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won agaist high rated player'
        elif row['winner_loser_elo_diff'] > 0 and row['winner_loser_elo_diff'] < 50:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against Similar Rating'

        elif row['winner_loser_elo_diff'] <= -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against very high rated player'
        elif row['winner_loser_elo_diff'] <= -50 and row['winner_loser_elo_diff'] > -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won agaist high rated player'
        elif row['winner_loser_elo_diff'] < 0 and row['winner_loser_elo_diff'] > -50:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against Similar Rating'
            

    elif row['white'] == 'Jay_1999' and row['result'] == 'Lost':
        if row['winner_loser_elo_diff'] >= 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against very high rated player'
        elif row['winner_loser_elo_diff'] >= 50 and row['winner_loser_elo_diff'] < 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost agaist high rated player'
        elif row['winner_loser_elo_diff'] >= 0 and row['winner_loser_elo_diff'] < 50:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against Similar Rating'
        if row['winner_loser_elo_diff'] <= -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against very high rated player'
        elif row['winner_loser_elo_diff'] <= -50 and row['winner_loser_elo_diff'] > -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost agaist high rated player'
        elif row['winner_loser_elo_diff'] < 0 and row['winner_loser_elo_diff'] > -50:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against Similar Rating'
        # if row['winner_loser_elo_diff'] >= 70:
        #     df.at[i, 'winner_loser_elo_diff'] = 'Lost against very high rated player'
        # elif row['winner_loser_elo_diff'] >= 30 and row['winner_loser_elo_diff'] <= 70:
        #     df.at[i, 'winner_loser_elo_diff'] = 'Lost agaist high rated player'
        # elif row['winner_loser_elo_diff'] >= 0 and row['winner_loser_elo_diff'] <= 30:
        #     df.at[i, 'winner_loser_elo_diff'] = 'Lost against Similar Rating'

    elif row['black'] == 'Jay_1999' and row['result'] == 'Won':
        # if row['winner_loser_elo_diff'] <= -100:
        #     df.at[i, 'winner_loser_elo_diff'] = 'Won against very high rated player (Great Victory)'
        # elif row['winner_loser_elo_diff'] <= -50 and row['winner_loser_elo_diff'] > -100:
        #     df.at[i, 'winner_loser_elo_diff'] = 'Won agaist high rated player (Excellent Win)'
        # elif row['winner_loser_elo_diff'] <= 0 and row['winner_loser_elo_diff'] > -50:
        #     df.at[i, 'winner_loser_elo_diff'] = 'Won against Similar Rating (Good Win)'
        if row['winner_loser_elo_diff'] >= 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against very high rated player'
        elif row['winner_loser_elo_diff'] >= 50 and row['winner_loser_elo_diff'] < 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won agaist high rated player'
        elif row['winner_loser_elo_diff'] >= 0 and row['winner_loser_elo_diff'] < 50:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against Similar Rating'
        elif row['winner_loser_elo_diff'] <= -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against very high rated player'
        elif row['winner_loser_elo_diff'] <= -50 and row['winner_loser_elo_diff'] > -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Won agaist high rated player'
        elif row['winner_loser_elo_diff'] < 0 and row['winner_loser_elo_diff'] > -50:
            df.at[i, 'winner_loser_elo_diff'] = 'Won against Similar Rating'


    
    elif row['black'] == 'Jay_1999' and row['result'] == 'Lost':
        if row['winner_loser_elo_diff'] <= -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against very high rated player'
        elif row['winner_loser_elo_diff'] <= -50 and row['winner_loser_elo_diff'] > -100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost agaist high rated player'
        elif row['winner_loser_elo_diff'] <= 0 and row['winner_loser_elo_diff'] > -50:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against Similar Rating'
        elif row['winner_loser_elo_diff'] >= 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against very high rated player'
        elif row['winner_loser_elo_diff'] >= 50 and row['winner_loser_elo_diff'] < 100:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost agaist high rated player'
        elif row['winner_loser_elo_diff'] > 0 and row['winner_loser_elo_diff'] < 50:
            df.at[i, 'winner_loser_elo_diff'] = 'Lost against Similar Rating'
    else:
        df.at[i, 'winner_loser_elo_diff'] = "Draw Game"



In [26]:
df['winner_loser_elo_diff'].value_counts()

Won against Similar Rating             675
Lost against Similar Rating            654
Won against very high rated player     632
Lost against very high rated player    300
Won agaist high rated player           282
Lost agaist high rated player          245
Draw Game                              116
0                                        7
Name: winner_loser_elo_diff, dtype: int64

In [27]:
df['variant'].value_counts()

Standard         2880
From Position      31
Name: variant, dtype: int64

In [30]:
df.columns

Index(['event', 'date_played', 'white', 'black', 'result', 'white_elo',
       'white_rating_diff', 'black_elo', 'black_rating_diff', 'winner',
       'winner_elo', 'loser', 'loser_elo', 'winner_loser_elo_diff', 'eco',
       'termination', 'time_control', 'utc_date', 'utc_time', 'variant',
       'Opening', 'myrating', 'hour', 'time_of_day', 'type of game'],
      dtype='object')

In [29]:
#df.to_csv('Jay_Games.csv')